# Getting started with OpenCV
Install the latest versions of OpenCV and imutils (used to simplify code) depending on your OS and IDE.

We will get straight into coding, explaining in each line its function.

In [1]:
import imutils
import cv2
# %matplotlib auto # to avoid not responding images windows

image = cv2.imread("insomnio.jpg")
(h, w, d) = image.shape
# in any array 3D you have the rows (height), columns (width) and depth(number of channels, if color=3)
# that is how images are represented 
print(f"height={h}, width={w}, depth={d}")

cv2.imshow("Image", image)
# show image in new window
cv2.waitKey(0)
# waiting for clicking the image and push any key to continue execution
# cv2.destroyAllWindows(); cv2.waitKey(1) # use it to show image and when clicked destroy it

height=400, width=600, depth=3


-1

Generally, the size of an image is presented as width (columns) x height (rows), e.g., 400x600=240000 px. Each pixel of a BW image has a value corresponding to its brightness in a grayscale bar, going from 0 to 255 (= 8bits resolution). In color images, let's start with the RGB space, which for historical reasons is used as BGR in opencv. It is represented as a 3-tuple with 256^3=16777216 combinations of colors.

In [2]:
(B,G,R) = image[40,200]
print(f"R={R}, G={G}, B={B}")

R=108, G=228, B=245


### Array slicing
The idea is to obtain a Region of Interest (ROI) for further processing. This action is can be automatically performed with deep learning or object tracking.

In [3]:
roi = image[0:170, 22:200]
# slicing the rows and the columns
cv2.imshow("ROI", roi)
cv2.waitKey(0)

-1

### Image resizing
To get a squared (or any other shaped) image, reduce the number of pixels for faster computation, fit size to specific case...

In [4]:
# First we resize the image without accounting for the aspect ratio
resized = cv2.resize(image, (800, 200)) # new width and height
cv2.imshow("Fixed Resizing", resized)
cv2.waitKey(0)

# to maintain aspect ratio is neccesary to compute the proportion and resize accordingly
# with imutils a wrapper function does the job
resized = imutils.resize(image, width=300) # provide width or height
cv2.imshow("Imutils Resize", resized)
cv2.waitKey(0)

-1

### Image rotation
Formally, we will need to calculate the point of rotation, in this case the center of the image. The rotation matrix is constructed with this information. Next we apply an affine transformation, a matrix multiplication (linear transformation) followed by a vector addition (translation), more specific, rotations (linear transformation), translations (vector addition) or scale operations (linear transformation).

Again imutils save us tedious work.

In [5]:
rotated = imutils.rotate(image, -45) # clockwise negative
cv2.imshow("Imutils Rotation", rotated)
cv2.waitKey(0)

# the image is cropped due to opencv not caring about what we actually did to the image
# this function preserves the whole size
rotated = imutils.rotate_bound(image, 45)
cv2.imshow("Imutils Bound Rotation", rotated)
cv2.waitKey(0)

-1

### Smoothing
Remove the high frequency content of images (like sharp edges) is useful for some algorithms or any other pipeline.

In [6]:
blurred = cv2.GaussianBlur(image, (11, 11), 0)
# gaussian with 11x11 kernel with 0 standard deviation in both x-y
cv2.imshow("Blurred", blurred)
cv2.waitKey(0)

-1

### Drawing
Any drawing operation is performed in-place (the original image is altered), therefore it is important to remember to create a copy and operate on this image.

In [12]:
# draw a rectangle on the copied image
output = image.copy()
cv2.rectangle(output, (22, 0), (200, 170), (0, 0, 255), 2)
# the position is defined as (x,y) in the in the top-left (origin) and bottom-right corners, forget the matrix notation
# the color in BGR and the line thickness (negative value filled rectangle)
cv2.circle(output, (300, 200), 20, (255, 0, 0), -1)
# center coordinates, radius in px, color (blue), thickness (filled)
cv2.line(output, (60, 80), (100, 300), (0, 0, 255), 5)
# line beginning and end coordinates, color and thickness
cv2.putText(output, "Drawing like a pro", (300, 250), 
	cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 100), 2)
# starting position,font used (generally sans-serif), scale, color and letter thickness
cv2.imshow("Drawing", output)
cv2.waitKey(0)

-1

### Real image processing
Working on the terminal and processing the images depending on the final goal. A script must be created to interact with the terminal.

In [ ]:
# Reminder:
# to run script on python console use: run script_name args
# to run script on OS terminal use: python (or python3) script_name.py args

import argparse # terminal working
import imutils
import cv2

# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True,
	help="specify path to input image, relative or global")
args = vars(ap.parse_args())
# vars transform the args.namespace to a keyword that can be used in a function

# loading the input image via command line
image = cv2.imread(args["image"]) # args["image"] = args.image
cv2.imshow("Image", image)
cv2.waitKey(0)
# converting to grayscale for edge detection and thresholding
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("Gray", gray)
cv2.waitKey(0)

### Edge detecion with Canny algorithm
edged = cv2.Canny(gray, 30, 150)
# min threshold, max threshold, and sobel kernel size (default 3)
# sobel filter: discrete differentiation operator, computing an approximation of the gradient of the image intensity function, enhancing high freq variations
cv2.imshow("Edged", edged)
cv2.waitKey(0)

### Thresholding
# used to remove dark or light areas and highlight some regions
# binary inverse threshold: in the image all pixel values less than 225
# to 255 (white; foreground) and all pixel values >= 225 to 255
# (black; background), thereby segmenting the image, critical step for finding contours
thresh = cv2.threshold(gray, 225, 255, cv2.THRESH_BINARY_INV)[1]
cv2.imshow("Thresh", thresh)
cv2.waitKey(0)

### Detecting contours
# using the thresholded image find the outlining
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
# simply finding white px in a copy of thresh
cnts = imutils.grab_contours(cnts)
# compatibility wrapper for previous versions
output = image.copy()
# make copy when drawing over images
for c in cnts:
	# draw each contour on the output image with a 3px thick purple
	# outline, then display the output contours one at a time
	cv2.drawContours(output, [c], -1, (240, 0, 159), 3)
#     cv2.imshow("Contours", output)
# 	cv2.waitKey(0)
    
text = f"{len(cnts)} objects found!"
cv2.putText(output, text, (10, 25),  cv2.FONT_HERSHEY_SIMPLEX, 0.7,
	(240, 0, 159), 2)
cv2.imshow("Contours", output)
cv2.waitKey(0)

### Erosions and dilations
# To reduce noise in thresholded binary images
# we apply erosions to reduce the size of foreground objects (eroding pixels)
# useful for removing some blobs or undesired objects
mask = thresh.copy()
mask = cv2.erode(mask, None, iterations=5)
# 5 iterations for reducing size
cv2.imshow("Eroded", mask)
# cv2.waitKey(0)

# Similarly dilations enlarge the ground and can combine foreground objects (nearby contours) when interested
mask = thresh.copy()
mask = cv2.dilate(mask, None, iterations=5)
cv2.imshow("Dilated", mask)
cv2.waitKey(0)

### Masking
# Mask out or hide regions we are not interested in, maybe to enchance some feature or the image.
mask = thresh.copy()
output = cv2.bitwise_and(image, image, mask=mask)
cv2.imshow("Output", output)
cv2.waitKey(0)